In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [3]:
text = "hello world! this is a simple rnn example."
chars = sorted(list(set(text)))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for i, c in enumerate(chars)}


In [4]:
vocab_size = len(chars)
print("Vocabulary size:", vocab_size)

Vocabulary size: 18


In [5]:
encoded = np.array([char2idx[c] for c in text])

In [6]:
seq_length = 10
x_data, y_data = [], []
for i in range(len(encoded) - seq_length):
    x_data.append(encoded[i:i+seq_length])
    y_data.append(encoded[i+seq_length])

x_data = np.array(x_data)
y_data = np.array(y_data)


In [7]:
from tensorflow.keras.layers import LSTM
model = Sequential([
    Embedding(vocab_size, 64, input_length=seq_length),
    LSTM(256),
    Dense(vocab_size, activation="softmax")
])


In [8]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 64)            1152      
                                                                 
 lstm (LSTM)                 (None, 256)               328704    
                                                                 
 dense (Dense)               (None, 18)                4626      
                                                                 
Total params: 334,482
Trainable params: 334,482
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(x_data, y_data, epochs=50, batch_size=32)

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 2.8913
Epoch 2/50
1/1 [==============================] - 0s 10ms/step - loss: 2.8815
Epoch 3/50
1/1 [==============================] - 0s 10ms/step - loss: 2.8710
Epoch 4/50
1/1 [==============================] - 0s 9ms/step - loss: 2.8590
Epoch 5/50
1/1 [==============================] - 0s 10ms/step - loss: 2.8445
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 2.8260
Epoch 7/50
1/1 [==============================] - 0s 9ms/step - loss: 2.8017
Epoch 8/50
1/1 [==============================] - 0s 10ms/step - loss: 2.7692
Epoch 9/50
1/1 [==============================] - 0s 11ms/step - loss: 2.7264
Epoch 10/50
1/1 [==============================] - 0s 10ms/step - loss: 2.6741
Epoch 11/50
1/1 [==============================] - 0s 9ms/step - loss: 2.6272
Epoch 12/50
1/1 [==============================] - 0s 9ms/step - loss: 2.6420
Epoch 13/50
1/1 [==============================] - 0s 9ms/step - los

In [10]:
def generate_text(seed_text, length=100):
    # If seed text is shorter, pad it (left side) with the first character of the dataset
    if len(seed_text) < seq_length:
        seed_text = seed_text.rjust(seq_length, text[0])
    # If seed text is longer, trim it
    elif len(seed_text) > seq_length:
        seed_text = seed_text[-seq_length:]

    input_seq = [char2idx[c] for c in seed_text]

    for _ in range(length):
        x = np.array([input_seq[-seq_length:]])  # always 10 chars
        preds = model.predict(x, verbose=0)[0]
        next_index = np.random.choice(len(preds), p=preds)
        input_seq.append(next_index)

    return ''.join(idx2char[i] for i in input_seq)

    

In [11]:
print("\nGenerated text:\n")
print(generate_text("hello world", length=100))   
print(generate_text("rnn example", length=100)) 
print(generate_text("hi", length=100))          



Generated text:

ello world! htstsiia  ispamllern  ..h srlee s s    ple empeeehor.a.ooi!si sthsipi    saplnerrn  rd enilpn a e.
nn example..ppxwtw o! hx.s is  epea lrep.wlri   ms s eleeer l .ladre     siilerlele n ips .r.een  se exx.neer.
hhhhhhhhhi   isssiplll  i   tsi llele i  diixm.lelmaresxd aimn e e lle..nelp i .neeaiellpe...end tp.  leninier


In [ ]:
# from tensorflow.keras.utils import get_file
# path = get_file("shakespeare.txt",
#                 "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
# text = open(path, 'r').read().lower()
# print(text[:1000])  # Print first 1000 characters

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to die than to famish?

all:
resolved. resolved.

first citizen:
first, you know caius marcius is chief enemy to the people.

all:
we know't, we know't.

first citizen:
let us kill him, and we'll have corn at our own price.
is't a verdict?

all:
no more talking on't; let it be done: away, away!

second citizen:
one word, good citizens.

first citizen:
we are accounted poor citizens, the patricians good.
what authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them let us revenge this with
our pikes, ere we become rakes: for the gods know i
speak this in hunger for bread, not in thirst for revenge.



In [13]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import get_file

In [14]:
path = get_file("shakespeare.txt",
                "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
text = open(path, 'r').read().lower()

In [15]:
chars = sorted(set(text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = np.array(chars)
text_as_int = np.array([char2idx[c] for c in text])


In [16]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)


In [17]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [19]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (dataset.shuffle(BUFFER_SIZE)
           .batch(BATCH_SIZE, drop_remainder=True))

vocab_size = len(chars)
embedding_dim = 256
rnn_units = 1024

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.SimpleRNN(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

In [22]:

model.compile(optimizer='adam', loss=loss)

EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/5
172/172 [==============================] - 21s 116ms/step - loss: 2.5115
Epoch 2/5
172/172 [==============================] - 22s 122ms/step - loss: 1.9961
Epoch 3/5
172/172 [==============================] - 19s 109ms/step - loss: 1.7804
Epoch 4/5
172/172 [==============================] - 20s 114ms/step - loss: 1.6473
Epoch 5/5
172/172 [==============================] - 21s 118ms/step - loss: 1.5612


In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.SimpleRNN(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])
model2.set_weights(model.get_weights())

In [ ]:
def generate_text(model, start_string, num_generate=500):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 1.0

    model.reset_states()
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

In [ ]:
print(generate_text(model2, start_string="hello: "))

Epoch 1/5
172/172 [==============================] - 19s 100ms/step - loss: 2.6331
Epoch 2/5
172/172 [==============================] - 17s 98ms/step - loss: 2.0982
Epoch 3/5
172/172 [==============================] - 18s 103ms/step - loss: 1.8741
Epoch 4/5
172/172 [==============================] - 18s 99ms/step - loss: 1.7177
Epoch 5/5
172/172 [==============================] - 19s 105ms/step - loss: 1.6168
hello: think you wefe and true: now them sheel'd discokn me a moss lord.

nurst:
ain; had young vistority.

jawnich:
i crund!

hastings:
if i have let my lords, and will demp wornd loblester
and were brout or, bust me no love achingers?

caunfoly verimation or make a sight these though romember:
i would hetcharie and desi:
livit not, it dough off with glow:
sor, how, lady frownill in youths.

first sentariag afording a.
this is to: sir, cannock,
he him like what jest thos in his city as youble some tha
